In [1]:
import yfinance as yf

stock_tickers = ['NVDA','TSLA','AAPL','AMZN']
database = yf.download(stock_tickers,start='2023-01-01',end ='2024-06-30')
database = database['Adj Close']

data = database.dropna().pct_change(1).dropna()
data

[*********************100%***********************]  4 of 4 completed


Ticker,AAPL,AMZN,NVDA,TSLA
Date,,,,
2023-01-04 00:00:00+00:00,0.010314,-0.007924,0.030318,0.051249
2023-01-05 00:00:00+00:00,-0.010605,-0.023726,-0.032816,-0.029039
2023-01-06 00:00:00+00:00,0.036794,0.035611,0.041640,0.024651
2023-01-09 00:00:00+00:00,0.004089,0.014870,0.051753,0.059349
2023-01-10 00:00:00+00:00,0.004456,0.028732,0.017981,-0.007681
...,...,...,...,...
2024-06-24 00:00:00+00:00,0.003133,-0.018564,-0.066840,-0.002350
2024-06-25 00:00:00+00:00,0.004468,0.004149,0.067564,0.026126
2024-06-26 00:00:00+00:00,0.019993,0.039015,0.002459,0.048145


In [2]:
import numpy as np
import pandas as pd

def sk_Criterion(weight,data):
    Lambda = 3
    W = 1
    Wbar = (1+0.25)/100
    portfolio_return = np.multiply(data,np.transpose(weight))
    portfolio_return = portfolio_return.sum(axis=1)
    mean = np.mean(portfolio_return,axis=0)
    std = np.std(portfolio_return,axis=0)
    skew = np.skew(portfolio_return)
    kurt = np.kurtosis(portfolio_return)
    
    mv_criterion = (Wbar**-1-Lambda)/1+Lambda + (Wbar**-Lambda)*W*mean - 0.5*Lambda*(W**2)*(std**2)*Wbar**(-1-Lambda) 
    
    criterion = mv_criterion + ((Lambda*(Lambda+1))/6)*(W**3)*(Wbar**-2-Lambda)*skew -((Lambda*(Lambda+1)*(Lambda+2))/24)*(W**4)*(Wbar**-3-Lambda)*kurt
    
    return -1*criterion
    

In [5]:
import numpy as np
from scipy.optimize import minimize

validation_score = 0.3
train_size = int(len(data)*(1-validation_score))
train_data = data.iloc[:train_size,:]
test_data = data.iloc[train_size:,:]

n = data.shape[1]
x0 = np.ones(n)

cons = ({'type':'eq','fun':lambda x:np.sum(abs(x))-1})
Bounds = [(0,1) for i in range(0,n)]
res = minimize(sk_Criterion,x0,args = train_data,method='SLSQP',constraints= cons,bounds=Bounds,options={'disp':True})
res_weight = res.x
res_weight


AttributeError: module 'numpy' has no attribute 'skew'